In [1]:
import re    # for regular expressions 
import nltk  # for text manipulation 
from nltk.corpus import stopwords
import string # for text manipulation 
import warnings
import pandas as pd # for data manipulation 
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore") #ignore warnings

# import ssl
# 
# try:
#     _create_unverified_https_context = ssl._create_unverified_context
# except AttributeError:
#     pass
# else:
#     ssl._create_default_https_context = _create_unverified_https_context
# 
# nltk.download()

import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

%matplotlib inline

./data/processed_full.csv
./data/processed.csv
./data/light_data.csv
./data/light.csv
./data/data.csv
./data/processed
./data/lightdata.csv


In [32]:
data = pd.read_csv("./data/data.csv",encoding='latin-1')
data.head()
data = data.sample(frac=1)
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "TweetText"]
data.columns = DATASET_COLUMNS
data.drop(['ids','date','flag','user'],axis = 1,inplace = True)
data = data[:30000]
data.loc[data['target'] == 4, 'target'] = 1
data.head()

,target,TweetText
817427,1,@tommcfly do you like Amsterdam?
865443,1,So excited for tonight!
1508665,1,Woo! Once i finish the game Diana is going to get me my tablets and sandwich. Such an awesome negotiator.
309038,0,please talk to me.
1088584,1,@shefaly Morning shefaly..the sun's always blinding..its the level of shade that you have that makes it bearable/nice


In [33]:
stopwords_list=stopwords.words('english')
from nltk.stem.porter import *
stemmer = PorterStemmer()
def remove_stopwords(text):
    clean_text=' '.join([word for word in text.split() if word not in stopwords_list])
    return clean_text

In [34]:
# data = pd.concat([positif_data,negative_data],axis = 0)
data['Clean_TweetText'] = data['TweetText'].str.replace("@", "")
data['Clean_TweetText'] = data['Clean_TweetText'].str.replace(r"http\S+", "")
data['Clean_TweetText'] = data['Clean_TweetText'].str.replace("[^a-zA-Z]", " ")
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda text : remove_stopwords(text.lower()))
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: x.split())
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: [stemmer.stem(i) for i in x])
data['Clean_TweetText'] = data['Clean_TweetText'].apply(lambda x: ' '.join([w for w in x]))

In [35]:
data = data.drop(labels=['TweetText'], axis=1)

In [36]:
data.head()

,target,Clean_TweetText
817427,1,tommcfli like amsterdam?
865443,1,excit tonight!
1508665,1,woo! finish game diana go tablet sandwich. awesom negotiator.
309038,0,pleas talk
1088584,1,shefali morn shefaly..th sun' alway blinding..it level shade make bearable/nic


In [37]:
data.to_csv('./data/processed_full.csv', sep=",",quotechar='"',index=False)

In [38]:
data.describe()

,target
count,30000.000000
mean,0.503400
std,0.499997
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000
